In [9]:
import tensorflow as tf
from tensorflow.python.saved_model import builder
import numpy as np

In [2]:
tf.__version__

'1.15.5'

In [3]:
model_export_dir = "simple_dag"

In [5]:
def simple_dag(input_1, input_2):
    o1 = input_1
    o2 = input_2

    for i in range(5):
        with tf.variable_scope(f"left/unit_{i}"):
            o1 = tf.layers.dense(inputs=o1, units=2, activation=tf.nn.relu,
                                kernel_initializer=tf.initializers.truncated_normal(), 
                                bias_initializer=tf.initializers.ones())

    for i in range(3):
        with tf.variable_scope(f"right/unit_{i}"):
            o2 = tf.layers.dense(inputs=o2, units=2, activation=tf.nn.relu,
                                kernel_initializer=tf.initializers.truncated_normal(), 
                                bias_initializer=tf.initializers.ones())
    
    ok = tf.concat([o1, o2], axis=1, name='concat')
    o = ok

    with tf.variable_scope("res"):
        for i in range(3):
            with tf.variable_scope(f"unit_{i}"):
                o = tf.layers.dense(inputs=o, units=4, activation=tf.nn.relu,
                                    kernel_initializer=tf.initializers.truncated_normal(), 
                                    bias_initializer=tf.initializers.ones())
        o = o + ok
    return o

In [6]:
# Save model to `SavedModel Protobuf`
with tf.Graph().as_default():
    input_1 = tf.placeholder(shape=[None, 2], dtype=tf.float32, name="a")
    input_2 = tf.placeholder(shape=[None, 5], dtype=tf.float32, name="b")
    output = simple_dag(input_1, input_2)
    save_builder = builder.SavedModelBuilder(model_export_dir)
    prediction_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
            inputs={
                "input_1": tf.saved_model.utils.build_tensor_info(input_1),
                "input_2": tf.saved_model.utils.build_tensor_info(input_2),
            },
            outputs={
                "output": tf.saved_model.utils.build_tensor_info(output),
            },
            method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
        )
    )
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        save_builder.add_meta_graph_and_variables(
            sess, [tf.saved_model.tag_constants.SERVING],
            signature_def_map={
                tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
                    prediction_signature 
            },
        )
        save_builder.save()

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: simple_dag/saved_model.pb


In [7]:
!ls -R simple_dag

simple_dag:
saved_model.pb	variables

simple_dag/variables:
variables.data-00000-of-00001  variables.index


Load the `SavedModelProtobuf`

In [10]:
with tf.Session(graph=tf.Graph()) as sess:
    meta_graph = tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], model_export_dir)
    signature = meta_graph.signature_def[tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
    input_ph = [sess.graph.get_tensor_by_name(tensor.name) for tensor in signature.inputs.values()]
    output_ph = [sess.graph.get_tensor_by_name(tensor.name) for tensor in signature.outputs.values()]
    # the first dimention of input placeholder is none, so we replace it with `4`
    feed_dict = { iph: np.random.uniform(size=[4] + iph.shape.as_list()[1:]).astype(np.float32) for iph in input_ph }
    output_array = sess.run(output_ph, feed_dict)
print(output_array)

INFO:tensorflow:Restoring parameters from simple_dag/variables/variables
[array([[8.744786 , 1.0333626, 1.2282178, 1.3905438],
       [8.801934 , 1.0430452, 1.2219393, 1.399497 ],
       [8.666908 , 1.0510672, 1.2604113, 1.3751616],
       [9.084023 , 1.0404501, 1.1223016, 1.5002646]], dtype=float32)]
